In [24]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

In [25]:
# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../hangang_total4_final.csv")
nonhangang_df = pd.read_csv("../../nonhangang_total4_final.csv")

In [26]:
# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

In [27]:
# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 CRI_score
gu = gu.merge(
    nonhangang_df[["자치구", "Score_Z_Transformed"]],
    left_on="SIGUNGU_NM",
    right_on="자치구",
    how="left"
).drop(columns=["자치구"])


In [28]:
# 한강공원: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)

# 공원명 Score_Z_Transformed merge
parks = parks.merge(
    hangang_df[["공원명", "Score_Z_Transformed"]],
    left_on="park_name",
    right_on="공원명",
    how="left"
).drop(columns=["공원명"])

In [29]:
# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["Score_Z_Transformed"].min(), parks["Score_Z_Transformed"].min())
all_max = max(gu["Score_Z_Transformed"].max(), parks["Score_Z_Transformed"].max())

cmap = plt.cm.Blues
norm = colors.Normalize(vmin=all_min, vmax=all_max)

# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["Score_Z_Transformed"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["Score_Z_Transformed"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "Score_Z_Transformed"],
        aliases=["자치구", "생태관광 점수"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["Score_Z_Transformed"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["Score_Z_Transformed"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "Score_Z_Transformed"],
        aliases=["공원", "생태관광 점수"]
    )
).add_to(m)

In [30]:
# 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("../result/total4_final_visualization.html")
print("total4_final_visualization.html 저장 완료")

total4_final_visualization.html 저장 완료
